# TMAE Analysis

Source: https://dadosabertos.aneel.gov.br/dataset/atendimento-ocorrencias-emergenciais

Last Update: 2026/02/17


## Libs:

In [2]:
! pip install --quiet requests 
! pip install --quiet pandas
! pip install --quiet numpy
! pip install --quiet seaborn
! pip install --quiet matplotlib
! pip install --quiet scikit-learn
! pip install --quiet scipy
import requests
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.patches import Patch
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
import datetime as dt

## Preparing Enviroment:

In [19]:
# Config - Df config:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None) 
pd.set_option("display.float_format", "{:.4f}".format)

In [ ]:
# Config - Image folder:
PROJECT_ROOT_DIR = "."
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images")
os.makedirs(IMAGES_PATH, exist_ok=True)

In [ ]:
# Function - Get data csv:
def get_data_csv(url_csv):
    df = pd.read_csv(url_csv, sep=";", encoding="latin1", decimal=',')
    return df

In [12]:
# Function - Get data API:
def get_data_api(resource_id):
    offset = 0
    total = []

    while True:
        url = f"https://dadosabertos.aneel.gov.br/api/3/action/datastore_search?resource_id={resource_id}&limit=10000&offset={offset}"
        result = requests.get(url)
        result.raise_for_status()    
        data_json = result.json()
        
        n = data_json['result']['records']

        if not n: 
            break  
        
        total.extend(n)
        offset += 10000
        print(f"Lines collected: {len(total)}")

        # Formação do df:
        df = pd.DataFrame(n)
    return df

In [10]:
# Function - Convert Columns:
def convert_dt_columns(df):
    # Dt Columns:
    columns_dt = [col for col in df.columns if col.upper().startswith('DT')]
    for col in columns_dt:
        df[col] = pd.to_datetime(df[col], errors='coerce')

    # Dat Columns:
    columns_dat = [col for col in df.columns if col.upper().startswith('DAT')]
    for col in columns_dat:
        df[col] = pd.to_datetime(df[col], errors='coerce')
    return df

## Getting Database:

In [20]:
# Principal database:
df = get_data_csv('https://dadosabertos.aneel.gov.br/dataset/dcdff65d-d173-4d2e-a3b9-84eec7702f06/resource/73b00e68-66b1-4a72-8d72-b7baab47048c/download/indicador-atendimento-emergencial.csv')
df.columns = df.columns.str.upper()
df.head()

,DATGERACAOCONJUNTODADOS,SIGAGENTE,NUMCNPJ,IDECONJUNDCONSUMIDORAS,DSCCONJUNDCONSUMIDORAS,SIGINDICADOR,ANOINDICE,NUMPERIODOINDICE,VLRINDICEENVIADO
0,05-02-2026,CPFL-PAULISTA,33050196000188.0000,25.0000,PIRACICABA,NDIACRI,2010,1,10.0000
1,05-02-2026,COELBA,15139629000194.0000,48.0000,MACAUBAS,NDIACRI,2010,1,15.0000
2,05-02-2026,ENEL CE,7047251000170.0000,49.0000,JUATAMA,NDIACRI,2010,1,0.0000
3,05-02-2026,NaN,19527639006601.0000,82.0000,TABULEIRO,NDIACRI,2010,1,3.0000
4,05-02-2026,EMR,19527639000158.0000,82.0000,TABULEIRO,NDIACRI,2010,1,3.0000


In [14]:
# Helpers - Databases - KPIs:
kpis = "e3724332-7de8-4273-8f0d-b20ebb91bf7c"
dfKPIs = get_data_api(kpis)
dfKPIs.columns = dfKPIs.columns.str.upper()
dfKPIs.head()

Lines collected: 468


,_ID,DATGERACAOCONJUNTODADOS,SIGINDICADOR,DSCINDICADOR
0,1,2022-08-17T00:00:00,AREA,"Área do conj., expressa em km2, correspondente a área geogr. e não a área elétr."
1,2,2022-08-17T00:00:00,AREAT,Área do conjunto em km2
2,3,2022-08-17T00:00:00,CM,Encargo de uso do sistema de distribuição aplicado à unidade cons. (mensal)
3,4,2022-08-17T00:00:00,CMA,Encargo de uso do sistema de distribuição aplicado à unidade consumidora (anual)
4,5,2022-08-17T00:00:00,CMM,"Consumo médio mensal, em MWh, média aritmética simples do último ano, excluin-"


In [15]:
# Helpers - Databases - Companies:
companies = "64250fc9-4f7a-4d97-b0d4-3c090e005e1c"
dfComp = get_data_api(companies)
dfComp.columns = dfComp.columns.str.upper()
dfComp.head()

Lines collected: 9736


,_ID,DATGERACAOCONJUNTODADOS,NUMCNPJ,SIGPESSOA,NOMRAZAOSOCIAL,IDCATIVO,IDCCOMERCIALIZACAO,IDCDISTRIBUICAO,IDCGERACAO,IDCTRANSMISSAO
0,1,2026-02-01,04890965000158,None,'' BG NORTE PETROLEO LTDA '',A,0,0,1,0
1,2,2026-02-01,32190231000100,None,2BR COMERCIALIZADORA DE ENERGIA LTDA,A,1,0,0,0
2,3,2026-02-01,03407182000108,None,2MS - ENGENHARIA LTDA,A,0,0,1,0
3,4,2026-02-01,36583766000193,None,2W COMERCIALIZADORA VAREJISTA DE ENERGIA S.A.,A,1,0,0,0
4,5,2026-02-01,08773135000100,None,2W ECOBANK S.A. - EM RECUPERACAO JUDICIAL,A,1,0,0,0


## Exploring Database:

In [16]:
# Database info:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6672090 entries, 0 to 6672089
Data columns (total 9 columns):
 #   Column                   Dtype  
---  ------                   -----  
 0   DATGERACAOCONJUNTODADOS  object 
 1   SIGAGENTE                object 
 2   NUMCNPJ                  float64
 3   IDECONJUNDCONSUMIDORAS   float64
 4   DSCCONJUNDCONSUMIDORAS   object 
 5   SIGINDICADOR             object 
 6   ANOINDICE                int64  
 7   NUMPERIODOINDICE         int64  
 8   VLRINDICEENVIADO         float64
dtypes: float64(3), int64(2), object(4)
memory usage: 458.1+ MB


In [17]:
# Removing unnecessary columns:
cols_to_drop = ['DATGERACAOCONJUNTODADOS']
df.drop(columns=cols_to_drop, inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6672090 entries, 0 to 6672089
Data columns (total 8 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   SIGAGENTE               object 
 1   NUMCNPJ                 float64
 2   IDECONJUNDCONSUMIDORAS  float64
 3   DSCCONJUNDCONSUMIDORAS  object 
 4   SIGINDICADOR            object 
 5   ANOINDICE               int64  
 6   NUMPERIODOINDICE        int64  
 7   VLRINDICEENVIADO        float64
dtypes: float64(3), int64(2), object(3)
memory usage: 407.2+ MB


In [21]:
# Database describe:
df.describe()

,NUMCNPJ,IDECONJUNDCONSUMIDORAS,ANOINDICE,NUMPERIODOINDICE,VLRINDICEENVIADO
count,6615234.0000,6672053.0000,6672090.0000,6672090.0000,6672090.0000
mean,14174070625003.6230,10643.3344,2010.3956,6.4132,99.7818
std,16596351633943.4863,4171.4301,7.6717,3.4899,297.6341
min,1229747000189.0000,1.0000,2000.0000,1.0000,-7112.4000
25%,4368898000106.0000,7815.0000,2004.0000,3.0000,5.3800
50%,8324196000181.0000,11113.0000,2009.0000,6.0000,40.8100
75%,15139629000194.0000,14158.0000,2017.0000,9.0000,106.8200
max,98042963000152.0000,17419.0000,2026.0000,12.0000,371168.0000


For this analysis, we will focus on the most recent five years of data, which requires filtering out earlier records from the dataset. We will also exclude data from 2026, as it is incomplete (only January is available).

In [23]:
# Removing unnecessary years:
df = df[df['ANOINDICE'] >= 2020]
df = df[df['ANOINDICE'] <= 2025]
df.describe()

,NUMCNPJ,IDECONJUNDCONSUMIDORAS,ANOINDICE,NUMPERIODOINDICE,VLRINDICEENVIADO
count,1184828.0000,1184828.0000,1184828.0000,1184828.0000,1184828.0000
mean,16013247124856.4824,14916.1878,2022.5088,6.3339,174.9401
std,18688199344202.6562,1422.5731,1.7104,3.5269,227.0472
min,1229747000189.0000,964.0000,2020.0000,1.0000,0.0000
25%,4895728000180.0000,13753.0000,2021.0000,3.0000,40.8900
50%,7522669000192.0000,14976.0000,2023.0000,6.0000,95.0000
75%,15413826000150.0000,16142.0000,2024.0000,9.0000,234.0000
max,98042963000152.0000,17419.0000,2025.0000,12.0000,78220.4700


Based on the number of records in each column, we can conclude that there are no missing values. Now, let us analyze the meaning of each KPI in the SIGINDICADOR column and select the ones that will be used in this study.